In [ ]:
import yfinance as yf  # Finansal veri indirmek için
import numpy as np  # Sayısal işlemler için
from sklearn.preprocessing import MinMaxScaler  # Veri ölçeklendirmek için
from tensorflow.keras.models import Sequential  # Model oluşturmak için
from tensorflow.keras.layers import LSTM, Dense, Dropout , Activation, BatchNormalization # Model katmanları
from tensorflow.keras.callbacks import EarlyStopping , ReduceLROnPlateau # Model eğitimini durdurmak için
from tensorflow.keras.regularizers import l1, l2

from sklearn.metrics import mean_squared_error  , mean_absolute_error , r2_score  # Hata ölçümü için
import mplfinance as mpf  # Finansal verileri görselleştirmek için
import pandas as pd  # Veri işleme için
import plotly.graph_objects as go

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
#veri indirme
parite_data = yf.download("ETH-USD", period="2000")
parite_data

In [ ]:
parite_df = parite_data[['Open', 'High', 'Low', 'Close' , 'Adj Close', 'Volume']]

In [ ]:

#girdiler tüm dataset kolonları , çıktılar ise 'Open', 'High', 'Low', 'Close' kolonu olacak şekilde zamanserisi oluşturulması
def create_time_series(data, time_steps, step):
    Xs, ys = [], []
    for i in range(0, len(data) - time_steps, step):
        v = data.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(data.iloc[i + time_steps].values)
    return np.array(Xs), np.array(ys)

#veri ölçeklendirme
scaler = MinMaxScaler()
parite_df = scaler.fit_transform(parite_df)

#zamanserisi oluşturma
TIME_STEPS = 30
STEP = 7
X, y = create_time_series(parite_df, TIME_STEPS, STEP)
print(X.shape, y.shape)

#veri setinin train ve test olarak ayrılması
train_size = int(len(X) * 0.80)
test_size = len(X) - train_size
X_train, X_test = X[0:train_size], X[train_size:len(X)]
y_train, y_test = y[0:train_size], y[train_size:len(y)]





In [ ]:
#modelin oluşturulması
model = Sequential()
model.add(LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(y_train.shape[1]))
model.add(Activation('linear'))
model.compile(loss='mse', optimizer='adam')

In [ ]:
#modelin eğitilmesi
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, verbose=1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=10), ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)])